<a href="https://colab.research.google.com/github/kwangwonland/kfq_ML_2023/blob/main/0718_%EB%94%A5%EB%9F%AC%EB%8B%9D_%ED%9B%88%EB%A0%A8_%EB%B0%8F_%ED%8F%89%EA%B0%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import models
from keras import layers

In [3]:
from keras.datasets import mnist

In [9]:
(x_train,y_train),(x_test,y_test)=mnist.load_data()
x_train=x_train.reshape(-1,784)
x_test=x_test.reshape(-1,784)
x_train=x_train/255.0
x_test=x_test/255.0
x_train=x_train.astype('float32')
x_test=x_test.astype('float32')
x_train,x_val=x_train[10000:],x_train[:10000]
y_train,y_val=y_train[10000:],y_train[:10000]

In [11]:
def get_mnist_model():
    inputs=keras.Input(shape=(784,))
    hidden=layers.Dense(512,activation='relu')(inputs)
    hidden2=layers.Dropout(0,5)(hidden)
    outputs=layers.Dense(10,activation='softmax')(hidden2)
    model=keras.Model(inputs,outputs)
    return model

model=get_mnist_model()
# 모델 컴파일
model.compile(optimizer='rmsprop',loss=keras.losses.sparse_categorical_crossentropy,metrics=['accuracy'])
# 모델 학습
hist=model.fit(x_train,y_train,epochs=20,validation_data=(x_val,y_val))
# 모델 평가
test=model.evaluate(x_test,y_test)
# 모델 사용
pred=model.predict(x_test)

Epoch 1/20
1563/1563 [==============================] - 14s 9ms/step - loss: 0.2222 - accuracy: 0.9337 - val_loss: 0.1467 - val_accuracy: 0.9588
Epoch 2/20
1563/1563 [==============================] - 12s 8ms/step - loss: 0.0928 - accuracy: 0.9725 - val_loss: 0.0888 - val_accuracy: 0.9738
Epoch 3/20
1563/1563 [==============================] - 13s 9ms/step - loss: 0.0629 - accuracy: 0.9805 - val_loss: 0.0837 - val_accuracy: 0.9770
Epoch 4/20
1563/1563 [==============================] - 12s 7ms/step - loss: 0.0463 - accuracy: 0.9857 - val_loss: 0.0863 - val_accuracy: 0.9772
Epoch 5/20
1563/1563 [==============================] - 12s 7ms/step - loss: 0.0344 - accuracy: 0.9894 - val_loss: 0.0900 - val_accuracy: 0.9783
Epoch 6/20
1563/1563 [==============================] - 12s 8ms/step - loss: 0.0256 - accuracy: 0.9925 - val_loss: 0.0874 - val_accuracy: 0.9771
Epoch 7/20
1563/1563 [==============================] - 12s 8ms/step - loss: 0.0200 - accuracy: 0.9941 - val_loss: 0.0839 - val_ac

#  사용자 평가 지표

In [32]:
class RootMeanSquaredError(keras.metrics.Metric):
  def __init__(self,name='rmse',**kwargs):
    super().__init__(name=name,**kwargs)
    self.mse_sum = self.add_weight(name='mse_sum',initializer='zeros')
    self.total_samples = self.add_weight(
        name = 'total_samples',
        initializer = 'zeros',dtype='int32'
    )
  def update_state(self,y_true,y_pred,sample_weight=None):
    y_true = tf.one_hot(y_true,depth=tf.shape(y_pred)[1])
    mse = tf.reduce_sum(tf.square(y_true - y_pred))
    self.mse_sum.assign_add(mse)
    num_samples = tf.shape(y_pred)[0]
    self.total_samples.assign_add(num_samples)
  def result(self):
    return tf.sqrt(self.mse_sum/tf.cast(self.total_samples,tf.float32))
  def reset_state(self):
    self.mse_sum.assign(0.)
    self.total_samples.assign(0)

In [33]:
model = get_mnist_model()
#모델 컴파일
model.compile(optimizer='rmsprop',
              loss = keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy',RootMeanSquaredError()])

In [34]:
model.fit(x_train,y_train,epochs=20,validation_data=(x_val,y_val))

Epoch 1/20
1563/1563 [==============================] - 13s 8ms/step - loss: 0.2176 - accuracy: 0.9361 - rmse: 7.2349 - val_loss: 0.1185 - val_accuracy: 0.9656 - val_rmse: 7.3756
Epoch 2/20
1563/1563 [==============================] - 13s 8ms/step - loss: 0.0913 - accuracy: 0.9731 - rmse: 7.3869 - val_loss: 0.0950 - val_accuracy: 0.9728 - val_rmse: 7.4130
Epoch 3/20
1563/1563 [==============================] - 12s 8ms/step - loss: 0.0628 - accuracy: 0.9807 - rmse: 7.4161 - val_loss: 0.0867 - val_accuracy: 0.9764 - val_rmse: 7.4274
Epoch 4/20
1563/1563 [==============================] - 13s 8ms/step - loss: 0.0450 - accuracy: 0.9866 - rmse: 7.4278 - val_loss: 0.0786 - val_accuracy: 0.9788 - val_rmse: 7.4374
Epoch 5/20
1563/1563 [==============================] - 12s 8ms/step - loss: 0.0360 - accuracy: 0.9886 - rmse: 7.4386 - val_loss: 0.0900 - val_accuracy: 0.9775 - val_rmse: 7.4420
Epoch 6/20
1563/1563 [==============================] - 12s 8ms/step - loss: 0.0263 - accuracy: 0.9922 - 

In [35]:
mcp=keras.callbacks.ModelCheckpoint('cp.keras',monitor='val_loss',save_best_only=True)
es=keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)

In [36]:
model = get_mnist_model()
#모델 컴파일
model.compile(optimizer='rmsprop',
              loss = keras.losses.sparse_categorical_crossentropy,
              metrics=[RootMeanSquaredError()])

hist=model.fit(x_train,y_train,epochs=20,validation_data=(x_val,y_val),callbacks=[mcp,es])

Epoch 1/20
1563/1563 [==============================] - 14s 9ms/step - loss: 0.2199 - rmse: 7.2347 - val_loss: 0.1197 - val_rmse: 7.3572
Epoch 2/20
1563/1563 [==============================] - 14s 9ms/step - loss: 0.0917 - rmse: 7.3873 - val_loss: 0.1067 - val_rmse: 7.4066
Epoch 3/20
1563/1563 [==============================] - 14s 9ms/step - loss: 0.0626 - rmse: 7.4159 - val_loss: 0.0909 - val_rmse: 7.4245
Epoch 4/20
1563/1563 [==============================] - 14s 9ms/step - loss: 0.0460 - rmse: 7.4307 - val_loss: 0.0909 - val_rmse: 7.4367
Epoch 5/20
1563/1563 [==============================] - 14s 9ms/step - loss: 0.0343 - rmse: 7.4421 - val_loss: 0.0887 - val_rmse: 7.4404
Epoch 6/20
1563/1563 [==============================] - 14s 9ms/step - loss: 0.0267 - rmse: 7.4454 - val_loss: 0.0942 - val_rmse: 7.4443
Epoch 7/20
1563/1563 [==============================] - 14s 9ms/step - loss: 0.0203 - rmse: 7.4502 - val_loss: 0.0881 - val_rmse: 7.4511
Epoch 8/20
1563/1563 [===================

In [37]:
model=keras.models.load_model('/content/cp.keras')

# 사용자 정의 콜백

In [31]:
class DemoCallBack(keras.callbacks.Callback):
  def on_epoch_begin(self,logs):
    print('eb')
  def on_epoch_end(self,logs):
    print('ee')
  def on_batch_begin(self,logs):
    print('bb')
  def on_batch_end(self,logs):
    print('be')
  def on_train_begin(self,logs):
    print('tb')
  def on_train_end(self,logs):
    print('te')

# 훈련 및 추론

In [44]:
metric=keras.metrics.SparseCategoricalAccuracy()
y_actual=[0,1,2]
pred=[[0.9,1,0.1],[0.45,0.44,0.11],[0.1,0.01,0.89]]

In [45]:
metric.update_state(y_actual,pred)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=3.0>

In [46]:
metrics=keras.metrics.MeanSquaredError()

In [49]:
y_actual=[87,95,68,104]
pred1=[78,82,93,91]
pred2=[80,90,75,200]

In [50]:
metrics.update_state(y_actual,pred1)
score=metrics.result()
print(score)
metrics.update_state(y_actual,pred2)
score=metrics.result()
print(score)

tf.Tensor(952.25, shape=(), dtype=float32)
tf.Tensor(1297.875, shape=(), dtype=float32)
